#  SyntaxCutter kasutusnäited

##  Stanza 

In [1]:
# Stanza tagger

import stanza
from estnltk.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

#model_path = ".../estnltk-version_1.6/estnltk/taggers/syntax/stanza_tagger/stanza_resources"
model_path = '/Users/rabauti/stanza-models/stanza_recources'
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)


# SyntaxCutter
from syntaxCutter.cutter import deprelCutter


## Conllu failid 

In [2]:
#conllu reader
from syntaxCutter.readerСonllu import reader

#kataloog conllu failidega
conllu_folder ='/Users/rabauti/Downloads/conllu_small'

oReader = reader.Reader(path=conllu_folder)

oCutter = deprelCutter.Cutter(stanza_tagger=stanza_tagger, Reader=oReader)
oCutter.cut(deprel='acl:relcl')


2022-07-07 20:49:54,367 - Cutter - INFO - Start cutting deprel: acl:relcl


2022-07-07 20:50:05,870 - Cutter - INFO - Done cutting deprel  acl:relcl .
2022-07-07 20:50:05,871 - Cutter - INFO - Stats stored in ./result/aclrelcl_20220707-204954/stats.tsv .


## Andmebaasist 

In [8]:
from syntaxCutter.readerDB import reader

oReader = reader.Reader(pgpass_file='~/.pgpass',\
                          schema='estonian_text_corpora',\
                          role='estonian_text_corpora_read',\
                          temporary=False,\
                          collectionName='koondkorpus_base_subset_of_5000_v2')

oCutter = deprelCutter.Cutter(stanza_tagger=stanza_tagger, Reader=oReader)

oCutter.cut(deprel='acl:relcl')

INFO:storage.py:42: connecting to host: 'postgres.keeleressursid.ee', port: '5432', dbname: 'estonian-text-corpora', user: 'zummy'
INFO:storage.py:58: schema: 'estonian_text_corpora', temporary: False, role: 'estonian_text_corpora_read'


2022-07-07 23:35:53,410 - Cutter - INFO - Start cutting deprel: acl:relcl
collection_id: 4999: 100%|##########| 5000/5000 [59:23<00:00,  1.40doc/s]
2022-07-08 00:35:17,502 - Cutter - INFO - Done cutting deprel  acl:relcl .
2022-07-08 00:35:17,503 - Cutter - INFO - Stats stored in ./result/aclrelcl_20220707-233553/stats.tsv .


## TSV failist

In [5]:
from syntaxCutter.readerTSV import reader

TSV_path = '/Users/rabauti/repos/estnltk-model-training/collocation_net/data_extraction/from_tsv/koondkorpus_base_subset_of_5000_v2.tsv'
oReader = reader.Reader(file=TSV_path)
oCutter = deprelCutter.Cutter(stanza_tagger=stanza_tagger, Reader=oReader)
oCutter.cut(deprel='acl:relcl')

2022-07-07 22:45:04,895 - Cutter - INFO - Start cutting deprel: acl:relcl
2022-07-07 22:45:04,898 - Reader - INFO - Reading sentences in progress.


2022-07-07 23:27:55,968 - Reader - INFO - Reading sentences done.
2022-07-07 23:27:55,970 - Cutter - INFO - Done cutting deprel  acl:relcl .
2022-07-07 23:27:55,971 - Cutter - INFO - Stats stored in ./result/aclrelcl_20220707-224504/stats.tsv .


## Kohandamine

In [6]:
# Filtreerib korpusest välja ainult hüüumärgiga lõppevad laused. (esialgse morf märgenduse põhjal) 
# Teeb määratud Stanza mudeliga uue analüüsi.
# Ignoreerib lauseid, milles on "conj" märgendiga sõnad.
# Lõikab välja deprel = 'nmod' tipud
# Teeb lühendatud lausele uuesti Stanza mudeliga analüüsi
# Salvestab konserveerunud laused, muutunud laused, laused, millest polnud midagi eemaldada 
# Salvestab üldise statistika


from syntaxCutter.sentence import sentence
class myCustomCutter(deprelCutter.Cutter):

    # funktsioon, kas esialgse süntaksiga lauset filtreerida välja või mitte
    # True puhul lauset ignoreeritakse
    def pre_filter(self, G, deprel):
        #ignoreeri kõiki lauseid, mis ei lõpe hüüumärgiga
        return not (sentence.Sentence.get_prop(G, 'lemma'))[-1:][0] == '!'
        

    # funktsioon, kas defineeritud Stanza mudeliga üle analüüsitud lauset filtreerida välja või mitte
    # True puhul lauset ignoreeritakse
    def filter(self, G, deprel):
        #ignoreeri kõiki lauseid, kus on sees sidesõna
        return  'conj' in sentence.Sentence.get_prop(G, 'deprel')
        #return False

    #oma funktsioon tippude eemaldamiseks
    def cutSentence(self, G, deprel):
        return sentence.Sentence.remove_deprel(G, 'nmod')

TSV_path = '/Users/rabauti/repos/estnltk-model-training/collocation_net/data_extraction/from_tsv/koondkorpus_base_subset_of_5000_v2.tsv'
oReader = reader.Reader(file=TSV_path)

oCutter = myCustomCutter(stanza_tagger=stanza_tagger, Reader=oReader)

oCutter.cut(deprel='mycustomcut')


2022-07-07 23:28:53,322 - Cutter - INFO - Start cutting deprel: mycustomcut
2022-07-07 23:28:53,326 - Reader - INFO - Reading sentences in progress.


2022-07-07 23:33:55,650 - Reader - INFO - Reading sentences done.
2022-07-07 23:33:55,653 - Cutter - INFO - Done cutting deprel  mycustomcut .
2022-07-07 23:33:55,653 - Cutter - INFO - Stats stored in ./result/mycustomcut_20220707-232853/stats.tsv .


## TSV fail

Kood TSV faili genereerimiseks asub siin:

https://github.com/estnltk/estnltk-model-training/blob/main/collocation_net/data_extraction/from_tsv/collect_texts_db_tsv.ipynb
